<a href="https://colab.research.google.com/github/vaibhavrouduri/Thesis-Work/blob/main/Vaibhav_Code/SC-WEAT/SC_WEAT_WEFAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
import time
from itertools import filterfalse
from scipy import spatial
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from gensim.scripts.glove2word2vec import glove2word2vec

# Glove Common Crawl

In [ ]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip # Retrieves content from the given URL
!unzip -q glove.840B.300d.zip # Unzips the file and returns the .txt file which can be used for analysis

In [ ]:
embeddings_glove = {} # Empty dictionary in which the words in the vocabulary and the corresponding vector will be added
with open("glove.840B.300d.txt") as f:
    for line in f: # Each line in the file contains the token followed by the vector representation of that token trained in glove in 300 dimensions separated by a space. Each dimension of the vector is separated by a space as well
        word, coefs = line.split(maxsplit=1) # Splits the line as described above at the first space, hence the token gets separated from the vector. The token is stored in 'word' and the vector is stored in 'coefs'
        coefs = np.fromstring(coefs, "f", sep=" ") # Returns the vector representation in the form of an array which can now be used for analysis (cosine similarity, etc)
        embeddings_glove[word] = coefs # Add the token and its corresponding vector into the dictionary as key and value pair. Now we can call the vector of any token in the vocabulary via the dictionary

# Glove Wikipedia

In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip # Retrieves content from the given URL
!unzip -q glove.6B.zip # Unzips the file and returns the .txt file which can be used for analysis. In this case, returns 4 files, having 50, 100, 200, and 300 dimensional vectors. We are choosing to use the 300 dimensional vectors

--2021-11-29 18:50:43--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-29 18:50:43--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-29 18:50:44--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [3]:
embeddings_glove_wiki_gw = {} # Empty dictionary in which the words in the vocabulary and the corresponding vector will be added
with open("glove.6B.300d.txt") as f:
    for line in f: # Each line in the file contains the token followed by a the vector representation of that token trained in glove in 300 dimensions separated by a space. Each dimension of the vector is separated by a space as well
        word, coefs = line.split(maxsplit=1) # Splits the line as described above at the first space, hence the token gets separated from the vector. The token is stored in 'word' and the vector is stored in 'coefs'
        coefs = np.fromstring(coefs, "f", sep=" ") # Returns the vector representation in the form of an array which can now be used for analysis (cosine similarity, etc)
        embeddings_glove_wiki_gw[word] = coefs # Add the token and its corresponding vector into the dictionary as key and value pair. Now we can call the vector of any token in the vocabulary via the dictionary

# Fastext Common Crawl

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip # Retrieves content from the given URL
!unzip -q crawl-300d-2M.vec.zip # Unzips the file and returns the .vec file which can be used for analysis

In [ ]:
embeddings_fastext_cc = {}
with open("crawl-300d-2M.vec") as f:
    for line in f: # Each line in the file contains the token followed by a the vector representation of that token trained in glove in 300 dimensions separated by a space. Each dimension of the vector is separated by a space as well
        word, coefs = line.split(maxsplit=1) # Splits the line as described above at the first space, hence the token gets separated from the vector. The token is stored in 'word' and the vector is stored in 'coefs'
        coefs = np.fromstring(coefs, "f", sep=" ") # Returns the vector representation in the form of an array which can now be used for analysis (cosine similarity, etc)
        embeddings_fastext_cc[word] = coefs # Add the token and its corresponding vector into the dictionary as key and value pair. Now we can call the vector of any token in the vocabulary via the dictionary

# Fastext Wikipedia

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip # Retrieves content from the given URL
!unzip -q wiki-news-300d-1M.vec.zip # Unzips the file and returns the .vec file which can be used for analysis

In [ ]:
embeddings_fastext_wiki = {}
with open("wiki-news-300d-1M.vec") as f:
    for line in f: # Each line in the file contains the token followed by a the vector representation of that token trained in glove in 300 dimensions separated by a space. Each dimension of the vector is separated by a space as well
        word, coefs = line.split(maxsplit=1) # Splits the line as described above at the first space, hence the token gets separated from the vector. The token is stored in 'word' and the vector is stored in 'coefs'
        coefs = np.fromstring(coefs, "f", sep=" ") # Returns the vector representation in the form of an array which can now be used for analysis (cosine similarity, etc)
        embeddings_fastext_cc[word] = coefs # Add the token and its corresponding vector into the dictionary as key and value pair. Now we can call the vector of any token in the vocabulary via the dictionary

# word2vec

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
embeddings_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# SC-WEAT

In [4]:
def cos(v1, v2): 
  """Returns the cosine similarity of 2 vectors of same dimesionality.

    Arguments:
    v1 -- Vector 1
    v2 -- Vector 2
    """

  num = np.dot(v1, v2)
  denom = np.linalg.norm(v1) * np.linalg.norm(v2)
  return num/denom

In [5]:
def weat_swAB(w, A, B, embedding_type):  
  """Returns the association of a single word with the chosen attribute (eg. good/bad).

    Arguments:
    w -- The string(word) whose association with the chosen attribute you want to find
    A -- List of Attribute words representing 1 end of the spectrum (eg. for the attribute Good/Bad, the list of words representing 'Good')
    B -- List of Attribute words representing the other end of the spectrum (eg. for the attribute Good/Bad, the list of words representing 'Bad')
    embedding_type -- The dictionary for the corpus algorithm pair being used in the current analysis (eg. for glove with common crawl, use embeddings_glove)
    """
  A_list = []
  B_list = []

  for v in A:
    A_list.append(cos(embedding_type[w], embedding_type[v]))
  for v in B:
    B_list.append(cos(embedding_type[w], embedding_type[v]))

  A_array = np.array(A_list)
  B_array = np.array(B_list)

  return np.mean(A_array) - np.mean(B_array)

In [6]:
def wefat_ES(w, A, B, embedding_type):
  """Returns the effect size of a word with the chosen attribute (eg. good/bad).

    Arguments:
    w -- Word whose relation to the attribute is to be found
    A -- List of Attribute words representing 1 end of the spectrum (eg. for the attribute Good/Bad, the list of words representing 'Good')
    B -- List of Attribute words representing the other end of the spectrum (eg. for the attribute Good/Bad, the list of words representing 'Bad')
    embedding_type -- The dictionary for the corpus algorithm pair being used in the current analysis (eg. for glove with common crawl, use embeddings_glove)
    """
  A_B = A + B
  cos_A_B = []
  for x in A_B:
    cos_A_B.append(cos(embedding_type[w], embedding_type[x]))
  cos_A_B_array = np.array(cos_A_B)

  return weat_swAB(w, A, B, embedding_type)/np.std(cos_A_B_array, ddof = 1)

In [7]:
def random_permutation(targets, r):
  """Returns a random permutation of a fixed number of a chosed list.

    Arguments:
    targets -- The list whose permutation is to be generated
    r -- Size of the permutation
    """
  return random.sample(targets, r)

In [12]:
def p_value_wefat(w, A, B, embedding_type, iterations):
  """Returns the p-value given a word and the chosen attribute (eg. good/bad).

    Arguments:
    w -- Word whose relation to the attribute is to be found
    A -- List of Attribute words representing 1 end of the spectrum (eg. for the attribute Good/Bad, the list of words representing 'Good')
    B -- List of Attribute words representing the other end of the spectrum (eg. for the attribute Good/Bad, the list of words representing 'Bad')
    embedding_type -- The dictionary for the corpus algorithm pair being used in the current analysis (eg. for glove with common crawl, use embeddings_glove)
    """
   
  A_B = A + B
  size_of_permutation = len(A)
  permutations = []
  test_statistics_permutations = []
  condition_satisfied = []

  while iterations != 0:
    permutations.append(random_permutation(A_B, size_of_permutation))
    iterations = iterations - 1

  for Ai in permutations:
    Bi = filterfalse(lambda w:w in Ai, A_B)
    test_statistics_permutations.append(weat_swAB(w, Ai, Bi, embedding_type))

  overall_test_statistic = weat_swAB(w, A, B, embedding_type)

  condition_satisfied = [p > overall_test_statistic for p in test_statistics_permutations]

  condition_satisfied_array = np.array(condition_satisfied)

  return condition_satisfied_array.sum()/condition_satisfied_array.size

# Example

In [13]:
# Attribute Stimuli
Good = ["love", "cheer", "friend", "pleasure", "paradise", "splendid"]
Bad = ["abuse", "grief", "poison", "sadness", "pain", "bomb"]


In [14]:
wefat_ES('baffling', Good, Bad, embeddings_glove_wiki_gw)

-0.2980413

In [15]:
p_value_wefat('baffling', Good, Bad, embeddings_glove_wiki_gw, 1000)

0.691